In [4]:
from qaravan.tensorQ import op_action, rdm_from_sv, environment_state_prep
from qaravan.core import ChiralHeisenberg, vN_entropy, two_local_circ, RunContext, TFI
import numpy as np
from scipy.sparse.linalg import LinearOperator
from scipy.sparse.linalg import eigsh
import sys, pickle, torch

def interp_ham(row_layout, sp, theta): 
    j = 1 
    jx = j * np.cos(theta)
    jy = jx
    jz = jx
    jc = j * np.sin(theta)
    return ChiralHeisenberg(row_layout, jx, jy, jz, jc, sp=sp)

def ham_action(grouped_terms, sv, local_dim=2): 
    result = torch.zeros_like(sv, dtype=torch.complex128) if torch.is_tensor(sv) else np.zeros_like(sv, dtype=complex)
    for indices, mat in grouped_terms: 
        result += op_action(mat, indices, sv, local_dim=local_dim)
    return result

def ground(ham, ncv=10, quiet=False):
    grouped_terms = ham.grouped_terms()
    dim = 2**ham.num_sites

    def mv(v):
        if v.ndim == 2 and v.shape[1] == 1:
            v = v[:, 0]
        return ham_action(grouped_terms, v)

    def convtest(eval_, evec, resNorm):
        print(f"Residual: {resNorm:.2e} for eigenvalue estimate: {eval_:.6f}")
        sys.stdout.flush()
        if np.abs(resNorm) < 1e-8: 
            return True
        return False
    
    ctest = convtest if not quiet else None
    H_linop = LinearOperator(shape=(dim, dim), matvec=mv, dtype=np.complex128)
    vals, vecs = eigsh(
        H_linop,
        k=1,
        which='SA',
        ncv=ncv,
    )
    return vals[0], vecs[:, 0]

In [6]:
from qaravan.core import brickwall_skeleton, two_local_circ
from qaravan.tensorQ import environment_state_prep

row_layout = [{'num_cells': 3, 'shift': 0.0}, 
             {'num_cells': 4, 'shift': -0.5}, 
             {'num_cells': 3, 'shift': 0.0}, 
             {'num_cells': 2, 'shift': 0.5}]
sp = [0,1,6,7,9,11,30,32,34,35,19,20]
ham = interp_ham(row_layout, sp, 0.35 * np.pi)
ground(ham)

KeyboardInterrupt: 